## CESM2 - LARGE ENSEMBLE (LENS2)
- The purpose of this notebook is to plot time series of the heat balance in the South Atlantic. 

#### Imports

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np 
import cf_xarray
from matplotlib.ticker import MaxNLocator
import cftime
import nc_time_axis
import matplotlib.pyplot as plt
import warnings, getpass, os
%matplotlib inline
from matplotlib.offsetbox import AnchoredText
import matplotlib.collections as collections
from scipy import stats

#### Load data

In [ ]:
# Meridional Heat Transport Difference, Total Surface Heat Flux, Heat Storage (from the heat balance and from the temperature trend)
ds_HB = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/heat_balance_components.nc')
# Meridional Heat Transport Difference, Total Surface Heat Flux, Heat Storage (from the heat balance and from the temperature trend) for PI Control
ds_HB_PI = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/heat_balance_components_PI_control.nc')
# Heat storage per layer: 0m-6000m
ds_HS = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/heat_storage_per_layer_0_6000m.nc')
# Heat storage per layer for PI Control: 0m-6000m 
ds_HS_PI = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/heat_storage_per_layer_0_6000m_PI_control.nc')

### Compute annual averages of the PI Control

In [ ]:
# PI CONTROL
MHTD_PI=ds_HB_PI.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31'))
# Total Surface Heat Flux
SHF_PI=ds_HB_PI.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31'))
# Heat Storage from the temperature difference
HS_TEND_TEMP_PI=ds_HB_PI.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31'))
# Heat Storage from the Heat Balance
HS_PI=ds_HB_PI.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31'))
# Heat storage per layer: 0m-1000m
HS_0_1000m_PI=ds_HS_PI.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31'))
# Heat storage per layer: 1000m-6000m
HS_1000_6000m_PI=ds_HS_PI.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31'))

In [ ]:
plt.plot(SHF_PI,label='SHF',color='blue')
plt.plot(MHTD_PI,label='MHTD',color='red')
plt.plot(HS_PI,label='HS1',color='navy')
plt.plot(HS_TEND_TEMP_PI,label='HS2',color='darkorange')
plt.plot(HS_0_1000m_PI,label='HS2: 0m-1000m',color='maroon')
plt.plot(HS_1000_6000m_PI,label='HS2: 1000m-6000m',color='darkgreen')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.grid(color='gray', linestyle='-', linewidth=0.7)
plt.ylabel('Heat Flux [PW]')
plt.xlabel('Times')
#plt.xlim(0,140)
#plt.hlines(0,0,140,color="black")
plt.savefig('Heat_balance_PI_control_terms.png',dpi=300,bbox_inches='tight')
plt.show()

### Calculate the linear trend of the PI Control

In [ ]:
def myfunc(x):
    return slope * x + intercept

In [ ]:
x=np.squeeze(range(0,len(MHTD_PI)))
slope, intercept, r, p, std_err = stats.linregress(x, MHTD_PI)
mymodel_MHTD_PI = list(map(myfunc, x))
slope_MHTD_PI=slope
m_MHTD_PI=slope_MHTD_PI*10 # per decade
p_MHTD_PI=p
r_MHTD_PI=r*r

x=np.squeeze(range(0,len(SHF_PI)))
slope, intercept, r, p, std_err = stats.linregress(x, SHF_PI)
mymodel_SHF_PI = list(map(myfunc, x))
slope_SHF_PI=slope
m_SHF_PI=slope_SHF_PI*10 # per decade
p_SHF_PI=p
r_SHF_PI=r*r

x=np.squeeze(range(0,len(HS_PI)))
slope, intercept, r, p, std_err = stats.linregress(x, HS_PI)
mymodel_HS_PI = list(map(myfunc, x))
slope_HS_PI=slope
m_HS_PI=slope_HS_PI*10 # per decade
p_HS_PI=p
r_HS_PI=r*r

x=np.squeeze(range(0,len(HS_TEND_TEMP_PI)))
slope, intercept, r, p, std_err = stats.linregress(x, HS_TEND_TEMP_PI)
mymodel_HS_TEND_TEMP_PI = list(map(myfunc, x))
slope_HS_TEND_TEMP_PI=slope
m_HS_TEND_TEMP_PI=slope_HS_TEND_TEMP_PI*10 # per decade
p_HS_TEND_TEMP_PI=p
r_HS_TEND_TEMP_PI=r*r

x=np.squeeze(range(0,len(HS_0_1000m_PI)))
slope, intercept, r, p, std_err = stats.linregress(x, HS_0_1000m_PI)
mymodel_HS_0_1000m_PI = list(map(myfunc, x))
slope_HS_0_1000m_PI=slope
m_HS_0_1000m_PI=slope_HS_0_1000m_PI*10 # per decade
p_HS_0_1000m_PI=p
r_HS_0_1000m_PI=r*r

x=np.squeeze(range(0,len(HS_1000_6000m_PI)))
slope, intercept, r, p, std_err = stats.linregress(x, HS_1000_6000m_PI)
mymodel_HS_1000_6000m_PI = list(map(myfunc, x))
slope_HS_1000_6000m_PI=slope
m_HS_1000_6000m_PI=slope_HS_1000_6000m_PI*10 # per decade
p_HS_1000_6000m_PI=p
r_HS_1000_6000m_PI=r*r

In [ ]:
nrows=1; ncols=3; height=8; width=22; alpha=0.3; pad=2

fig,ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(width,height),sharey=False)
#-----------------------------------------------------------------------------------------------------------------
# Plot 1
ds_HB_PI.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).plot.line(
    ax=ax[0],
    x='time',
    linewidth=2,
    color='red',
    label='MHTD_PI')
ax[0].plot(ds_HB_PI.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).coords['time'],mymodel_MHTD_PI,color='black',linestyle='dashed')
ds_HB_PI.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).plot.line(
    ax=ax[0],
    x='time',
    linewidth=2,
    color='blue',
    label='SHF_PI')
ax[0].plot(ds_HB_PI.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).coords['time'],mymodel_SHF_PI,color='black',linestyle='dashed')
ax[0].annotate(f'{m_SHF_PI:.8f} PW per decade', xy=(0.51, 0.97), color='blue',xycoords=ax[0].transAxes)
ax[0].annotate(f'(p-value={p_SHF_PI:.8f}, R2={r_SHF_PI:.8f})', xy=(0.48, 0.94), color='blue',xycoords=ax[0].transAxes)
ax[0].annotate(f'{m_MHTD_PI:.8f} PW per decade', xy=(0.51, 0.05), color='red',xycoords=ax[0].transAxes)
ax[0].annotate(f'(p-value={p_MHTD_PI:.8f}, R2={r_MHTD_PI:.8f})', xy=(0.48, 0.02), color='red',xycoords=ax[0].transAxes)
ax[0].set_ylim(0.1,0.7)
ax[0].set_xlim(ds_HB_PI.HS_TEND_TEMP.coords['time'][13199].values,ds_HB_PI.HS_TEND_TEMP.coords['time'][22186].values)
ax[0].set_ylabel('Heat Flux [PW]')
ax[0].set_xlabel('Time [Years]')
ax[0].grid(color='gray', linestyle='-', linewidth=0.7)
at = AnchoredText("A", prop=dict(size=15), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax[0].add_artist(at)
ax[0].legend(loc='lower left')
ax[0].set_title(None)

#-----------------------------------------------------------------------------------------------------------------
# Plot 2
ds_HB_PI.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).plot.line(
    ax=ax[1],
    x='time',
    linewidth=2,
    color='navy',
    label='HS1_PI')
ax[1].plot(ds_HB_PI.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).coords['time'],mymodel_HS_PI,color='black',linestyle='dashed')
ds_HB_PI.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).plot.line(
    ax=ax[1],
    x='time',
    linewidth=2,
    color='darkorange',
    label='HS2_PI')
ax[1].plot(ds_HB_PI.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).coords['time'],mymodel_HS_TEND_TEMP_PI,color='black',linestyle='dashed')
ax[1].annotate(f'{m_HS_PI:.8f} PW per decade', xy=(0.51, 0.97), color='navy',xycoords=ax[1].transAxes)
ax[1].annotate(f'(p-value={p_HS_PI:.8f}, R2={r_HS_PI:.8f})', xy=(0.48, 0.94), color='navy',xycoords=ax[1].transAxes)
ax[1].annotate(f'{m_HS_TEND_TEMP_PI:.8f} PW per decade', xy=(0.51, 0.05), color='darkorange',xycoords=ax[1].transAxes)
ax[1].annotate(f'(p-value={p_HS_TEND_TEMP_PI:.8f}, R2={r_HS_TEND_TEMP_PI:.8f})', xy=(0.48, 0.02), color='darkorange',xycoords=ax[1].transAxes)
ax[1].set_ylim(-0.3,0.3)
ax[1].set_xlim(ds_HB_PI.HS_TEND_TEMP.coords['time'][13199].values,ds_HB_PI.HS_TEND_TEMP.coords['time'][22186].values)
ax[1].set_ylabel('Heat Flux [PW]')
ax[1].set_xlabel('Time [Years]')
ax[1].grid(color='gray', linestyle='-', linewidth=0.7)
at = AnchoredText("B", prop=dict(size=15), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax[1].add_artist(at)
ax[1].legend(loc='lower left')
#ax[1].vlines(ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'][0].values,-0.35,0.35,linestyle='dashed',color="black")
ax[1].set_title(None)

#-----------------------------------------------------------------------------------------------------------------
# Plot 3
ds_HS_PI.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).plot.line(
    ax=ax[2],
    x='time',
    linewidth=2,
    color='maroon',
    label='HS2_PI: 0m-1000m')
ax[2].plot(ds_HS_PI.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).coords['time'],mymodel_HS_0_1000m_PI,color='black',linestyle='dashed')
ds_HS_PI.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).plot.line(
    ax=ax[2],
    x='time',
    linewidth=2,
    color='darkgreen',
    label='HS2_PI: 1000m-6000m')
ax[2].plot(ds_HS_PI.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','1849-12-31')).coords['time'],mymodel_HS_1000_6000m_PI,color='black',linestyle='dashed')
ax[2].annotate(f'{m_HS_0_1000m_PI:.8f} PW per decade', xy=(0.51, 0.97), color='maroon',xycoords=ax[2].transAxes)
ax[2].annotate(f'(p-value={p_HS_0_1000m_PI:.8f}, R2={r_HS_0_1000m_PI:.8f})', xy=(0.48, 0.94), color='maroon',xycoords=ax[2].transAxes)
ax[2].annotate(f'{m_HS_1000_6000m_PI:.8f} PW per decade', xy=(0.51, 0.05), color='darkgreen',xycoords=ax[2].transAxes)
ax[2].annotate(f'(p-value={p_HS_1000_6000m_PI:.8f}, R2={r_HS_1000_6000m_PI:.8f})', xy=(0.48, 0.02), color='darkgreen',xycoords=ax[2].transAxes)
ax[2].set_ylim(-0.3,0.3)
ax[2].set_xlim(ds_HS_PI.HS_1.coords['time'][13199].values,ds_HS_PI.HS_1.coords['time'][22186].values)
ax[2].set_ylabel('Heat Flux [PW]')
ax[2].set_xlabel('Time [Years]')
ax[2].grid(color='gray', linestyle='-', linewidth=0.7)
at = AnchoredText("C", prop=dict(size=15), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax[2].add_artist(at)
ax[2].legend(loc='lower left')
#ax[2].vlines(ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'][0].values,-0.3,0.3,linestyle='dashed',color="black")
ax[2].set_title(None)
fig.tight_layout(pad=pad)
plt.savefig('Heat_balance_1Y_PI_Control.png',dpi=300,bbox_inches='tight')
plt.show()

#### Linear trends
- We will calculate the linear trend only for the ssp370 period and average all members from the annual average

In [ ]:
# Meridional Heat Transport Difference
MHTD=ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).mean('member_id')-slope_MHTD_PI
# Total Surface Heat Flux
SHF=ds_HB.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).mean('member_id')-slope_SHF_PI
# Heat Storage from the temperature difference
HS_TEND_TEMP=ds_HB.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).mean('member_id')+slope_HS_TEND_TEMP_PI
# Heat Storage from the Heat Balance
HS=ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).mean('member_id')+slope_HS_PI
# Heat storage per layer: 0m-1000m
HS_0_1000m=ds_HS.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).mean('member_id')+slope_HS_0_1000m_PI
# Heat storage per layer: 1000m-6000m
HS_1000_6000m=ds_HS.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).mean('member_id')+slope_HS_1000_6000m_PI

- Calculating the trend from the polyfit method. Since you need to specify the dimension in xarray, you also need to do the following procedure, because xarray computes the trend per nanoseconds: [units/ns] * 1e9 [ns/s] * 60 [s/m] * 60 [m/h] * 24 [h/d]

- Calculating the trend from the stats method. 

In [ ]:
x=np.squeeze(range(0,len(MHTD)))
slope, intercept, r, p, std_err = stats.linregress(x, MHTD)
mymodel_MHTD = list(map(myfunc, x))
mymodel_MHTD=mymodel_MHTD-slope_MHTD_PI
m_MHTD=(slope-slope_MHTD_PI)*10 # per decade
p_MHTD=p
r_MHTD=r*r

x=np.squeeze(range(0,len(SHF)))
slope, intercept, r, p, std_err = stats.linregress(x, SHF)
mymodel_SHF = list(map(myfunc, x))
mymodel_SHF=mymodel_SHF-slope_SHF_PI
m_SHF=(slope-slope_SHF_PI)*10 # per decade
p_SHF=p
r_SHF=r*r

x=np.squeeze(range(0,len(HS)))
slope, intercept, r, p, std_err = stats.linregress(x, HS)
mymodel_HS = list(map(myfunc, x))
mymodel_HS = mymodel_HS+slope_HS_PI
m_HS=(slope+slope_HS_PI)*10 # per decade
p_HS=p
r_HS=r*r

x=np.squeeze(range(0,len(HS_TEND_TEMP)))
slope, intercept, r, p, std_err = stats.linregress(x, HS_TEND_TEMP)
mymodel_HS_TEND_TEMP = list(map(myfunc, x))
mymodel_HS_TEND_TEMP=mymodel_HS_TEND_TEMP+slope_HS_TEND_TEMP_PI
m_HS_TEND_TEMP=(slope+slope_HS_TEND_TEMP_PI)*10 # per decade
p_HS_TEND_TEMP=p
r_HS_TEND_TEMP=r*r

x=np.squeeze(range(0,len(HS_0_1000m)))
slope, intercept, r, p, std_err = stats.linregress(x, HS_0_1000m)
mymodel_HS_0_1000m = list(map(myfunc, x))
mymodel_HS_0_1000m=mymodel_HS_0_1000m+slope_HS_0_1000m_PI
m_HS_0_1000m=(slope+slope_HS_0_1000m_PI)*10 # per decade
p_HS_0_1000m=p
r_HS_0_1000m=r*r

x=np.squeeze(range(0,len(HS_1000_6000m)))
slope, intercept, r, p, std_err = stats.linregress(x, HS_1000_6000m)
mymodel_HS_1000_6000m = list(map(myfunc, x))
mymodel_HS_1000_6000m=mymodel_HS_1000_6000m+slope_HS_1000_6000m_PI
m_HS_1000_6000m=(slope+slope_HS_1000_6000m_PI)*10 # per decade
p_HS_1000_6000m=p
r_HS_1000_6000m=r*r

#### Calculate the highest and lowest value among all members to make the spread

In [ ]:
nrows=1; ncols=3; height=8; width=22; alpha=0.3; pad=2

fig,ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(width,height),sharey=False)
#-----------------------------------------------------------------------------------------------------------------
# Plot 1
spread_MHTD_max=ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).max(dim='member_id')
spread_MHTD_min=ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).min(dim='member_id')
ax[0].fill_between(ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').coords['time'].values,
                   spread_MHTD_max,spread_MHTD_min, facecolor='red',alpha=0.1)
spread_SHF_max=ds_HB.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).max(dim='member_id')
spread_SHF_min=ds_HB.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).min(dim='member_id')
ax[0].fill_between(ds_HB.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').coords['time'].values,
                   spread_SHF_max,spread_SHF_min, facecolor='blue',alpha=0.1)
ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').plot.line(
    ax=ax[0],
    x='time',
    linewidth=2,
    color='red',
    label='MHTD')
ax[0].plot(ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'],mymodel_MHTD,color='red',linestyle='dashed')
ds_HB.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').plot.line(
    ax=ax[0],
    x='time',
    linewidth=2,
    color='blue',
    label='SHF')
ax[0].plot(ds_HB.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'],mymodel_SHF,color='blue',linestyle='dashed')
ax[0].annotate(f'{m_SHF:.4f} PW per decade', xy=(0.51, 0.85), color='blue',xycoords=ax[0].transAxes)
ax[0].annotate(f'(p-value={p_SHF:.3f}, R2={r_SHF:.3f})', xy=(0.48, 0.82), color='blue',xycoords=ax[0].transAxes)
ax[0].annotate(f'{m_MHTD:.4f} PW per decade', xy=(0.51, 0.1), color='red',xycoords=ax[0].transAxes)
ax[0].annotate(f'(p-value={p_MHTD:.3f}, R2={r_MHTD:.3f})', xy=(0.48, 0.07), color='red',xycoords=ax[0].transAxes)
ax[0].set_ylim(0.1,0.7)
ax[0].set_xlim(ds_HB.HS_TEND_TEMP.coords['time'][1319].values,ds_HB.HS_TEND_TEMP.coords['time'][3010].values)
ax[0].set_ylabel('Heat Flux [PW]')
ax[0].set_xlabel('Time [Years]')
ax[0].grid(color='gray', linestyle='-', linewidth=0.7)
at = AnchoredText("A", prop=dict(size=15), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax[0].add_artist(at)
ax[0].legend(loc='lower left')
ax[0].vlines(ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'][0].values,0.1,0.7,linestyle='dashed',color="black")
ax[0].set_title(None)

#-----------------------------------------------------------------------------------------------------------------
# Plot 2
spread_HS_TEND_TEMP_max=ds_HB.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).max(dim='member_id')
spread_HS_TEND_TEMP_min=ds_HB.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).min(dim='member_id')
ax[1].fill_between(ds_HB.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').coords['time'].values,
                   spread_HS_TEND_TEMP_max,spread_HS_TEND_TEMP_min, facecolor='darkorange',alpha=0.1)

spread_HS_max=ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).max(dim='member_id')
spread_HS_min=ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).min(dim='member_id')
ax[1].fill_between(ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').coords['time'].values,
                   spread_HS_max,spread_HS_min, facecolor='navy',alpha=0.1)
ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').plot.line(
    ax=ax[1],
    x='time',
    linewidth=2,
    color='navy',
    label='HS1')
ax[1].plot(ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'],mymodel_HS,color='navy',linestyle='dashed')
ds_HB.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').plot.line(
    ax=ax[1],
    x='time',
    linewidth=2,
    color='darkorange',
    label='HS2')
ax[1].plot(ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'],mymodel_HS_TEND_TEMP,color='darkorange',linestyle='dashed')
ax[1].annotate(f'{m_HS:.4f} PW per decade', xy=(0.51, 0.85), color='navy',xycoords=ax[1].transAxes)
ax[1].annotate(f'(p-value={p_HS:.3f}, R2={r_HS:.3f})', xy=(0.48, 0.82), color='navy',xycoords=ax[1].transAxes)
ax[1].annotate(f'{m_HS_TEND_TEMP:.4f} PW per decade', xy=(0.51, 0.45), color='darkorange',xycoords=ax[1].transAxes)
ax[1].annotate(f'(p-value={p_HS_TEND_TEMP:.3f}, R2={r_HS_TEND_TEMP:.3f})', xy=(0.48, 0.42), color='darkorange',xycoords=ax[1].transAxes)
ax[1].set_ylim(-0.35,0.35)
ax[1].set_xlim(ds_HB.HS_TEND_TEMP.coords['time'][1319].values,ds_HB.HS_TEND_TEMP.coords['time'][3010].values)
ax[1].set_ylabel('Heat Flux [PW]')
ax[1].set_xlabel('Time [Years]')
ax[1].grid(color='gray', linestyle='-', linewidth=0.7)
at = AnchoredText("B", prop=dict(size=15), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax[1].add_artist(at)
ax[1].legend(loc='lower left')
ax[1].vlines(ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'][0].values,-0.35,0.35,linestyle='dashed',color="black")
ax[1].set_title(None)

#-----------------------------------------------------------------------------------------------------------------
# Plot 3
spread_HS_0_max=ds_HS.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).max(dim='member_id')
spread_HS_0_min=ds_HS.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).min(dim='member_id')
ax[2].fill_between(ds_HS.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').coords['time'].values,
                   spread_HS_0_max,spread_HS_0_min, facecolor='maroon',alpha=0.1)
spread_HS_1_max=ds_HS.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).max(dim='member_id')
spread_HS_1_min=ds_HS.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).min(dim='member_id')
ax[2].fill_between(ds_HS.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').coords['time'].values,
                   spread_HS_1_max,spread_HS_1_min, facecolor='darkgreen',alpha=0.1)
ds_HS.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').plot.line(
    ax=ax[2],
    x='time',
    linewidth=2,
    color='maroon',
    label='HS2: 0m-1000m')
ax[2].plot(ds_HS.HS_0.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'],mymodel_HS_0_1000m,color='maroon',linestyle='dashed')
ds_HS.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id').plot.line(
    ax=ax[2],
    x='time',
    linewidth=2,
    color='darkgreen',
    label='HS2: 1000m-6000m')
ax[2].plot(ds_HS.HS_1.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'],mymodel_HS_1000_6000m,color='darkgreen',linestyle='dashed')
ax[2].annotate(f'{m_HS_0_1000m:.4f} PW per decade', xy=(0.51, 0.85), color='maroon',xycoords=ax[2].transAxes)
ax[2].annotate(f'(p-value={p_HS_0_1000m:.3f}, R2={r_HS_0_1000m:.3f})', xy=(0.48, 0.82), color='maroon',xycoords=ax[2].transAxes)
ax[2].annotate(f'{m_HS_1000_6000m:.4f} PW per decade', xy=(0.51, 0.35), color='darkgreen',xycoords=ax[2].transAxes)
ax[2].annotate(f'(p-value={p_HS_1000_6000m:.3f}, R2={r_HS_1000_6000m:.3f})', xy=(0.48, 0.32), color='darkgreen',xycoords=ax[2].transAxes)
ax[2].set_ylim(-0.3,0.3)
ax[2].set_xlim(ds_HS.HS_1.coords['time'][1319].values,ds_HS.HS_1.coords['time'][3010].values)
ax[2].set_ylabel('Heat Flux [PW]')
ax[2].set_xlabel('Time [Years]')
ax[2].grid(color='gray', linestyle='-', linewidth=0.7)
at = AnchoredText("C", prop=dict(size=15), frameon=True, loc='upper left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax[2].add_artist(at)
ax[2].legend(loc='lower left')
ax[2].vlines(ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('2015-01-01','2100-12-31')).coords['time'][0].values,-0.3,0.3,linestyle='dashed',color="black")
ax[2].set_title(None)
fig.tight_layout(pad=pad)
plt.savefig('Heat_balance_1Y.png',dpi=300,bbox_inches='tight')
plt.show()

#### Correlation

In [ ]:
from scipy import signal

In [ ]:
ds1=ds_HB.MHTD.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id')
ds1=signal.detrend(ds1)

In [ ]:
ds3=ds_HB.HS.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id')
ds3=signal.detrend(ds3)

In [ ]:
ds2=ds_HB.SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id')
ds2=signal.detrend(ds2)

In [ ]:
ds4=ds_HB.HS_TEND_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1960-01-01','2100-12-31')).mean('member_id')
ds4=signal.detrend(ds4)

In [ ]:
plt.plot(ds1,label='SHF',color='blue')
plt.plot(ds2,label='MHTD',color='red')
plt.plot(ds3,label='HS1',color='navy')
plt.plot(ds4,label='HS2',color='darkorange')
plt.legend(loc="lower right", ncol=4)
plt.grid(color='gray', linestyle='-', linewidth=0.7)
plt.ylabel('Heat Flux [PW]')
plt.xlabel('Times')
plt.xlim(0,140)
plt.hlines(0,0,140,color="black")
plt.savefig('Heat_balance_1Y_correlations.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
np.corrcoef(ds1,ds2)

In [ ]:
np.corrcoef(ds1,ds3)

In [ ]:
np.corrcoef(ds2,ds3)

In [ ]:
np.corrcoef(ds4,ds3)